In [2]:
!pip install ipydeps

In [3]:
import ipydeps
ipydeps.pip(["keras-rl", "gym"])

In [4]:
!pip uninstall tensorflow

Uninstalling tensorflow-1.15.0:
  Would remove:
    /tensorflow-1.15.0/python3.6/tensorflow-1.15.0.dist-info/*
    /tensorflow-1.15.0/python3.6/tensorflow/*
    /tensorflow-1.15.0/python3.6/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [5]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 71kB/s 
     |████████████████████████████████| 3.2MB 35.0MB/s 
     |████████████████████████████████| 368kB 42.1MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.13.1 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0


In [6]:
import numpy as np
import gym
from gym import spaces
from gym.utils import seeding

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
env = gym.make('Blackjack-v0')
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [8]:
model = Sequential()
model.add(Flatten(input_shape=(1,3)))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                64        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
________________________________________________

In [0]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [10]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 8:37 - reward: 0.0000e+00WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


/root/.local/lib/python3.6/site-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 61s 6ms/step - reward: -0.1740
done, took 60.731 seconds


In [0]:
dqn.save_weights('dqn_blackjack_v0_weights.h5f', overwrite=True)

In [14]:
dqn.test(env, nb_episodes=100, visualize=False)

Testing for 100 episodes ...
Episode 1: reward: -1.000, steps: 1
Episode 2: reward: -1.000, steps: 2
Episode 3: reward: 1.000, steps: 3
Episode 4: reward: -1.000, steps: 2
Episode 5: reward: 1.000, steps: 2
Episode 6: reward: -1.000, steps: 3
Episode 7: reward: -1.000, steps: 1
Episode 8: reward: 1.000, steps: 2
Episode 9: reward: -1.000, steps: 1
Episode 10: reward: 1.000, steps: 2
Episode 11: reward: -1.000, steps: 2
Episode 12: reward: -1.000, steps: 1
Episode 13: reward: 1.000, steps: 1
Episode 14: reward: 0.000, steps: 1
Episode 15: reward: 0.000, steps: 3
Episode 16: reward: -1.000, steps: 1
Episode 17: reward: -1.000, steps: 2
Episode 18: reward: -1.000, steps: 1
Episode 19: reward: -1.000, steps: 2
Episode 20: reward: 0.000, steps: 1
Episode 21: reward: 1.000, steps: 2
Episode 22: reward: -1.000, steps: 1
Episode 23: reward: -1.000, steps: 2
Episode 24: reward: -1.000, steps: 2
Episode 25: reward: -1.000, steps: 2
Episode 26: reward: 1.000, steps: 1
Episode 27: reward: -1.000, 